In [2]:
import cv2
import dlib
import numpy as np
import os
import time

In [3]:
def get_rotation_matrix(landmarks):
    left_corner = np.array([landmarks.part(48).x, landmarks.part(48).y])
    right_corner = np.array([landmarks.part(54).x, landmarks.part(54).y])

    dy = right_corner[1] - left_corner[1]
    dx = right_corner[0] - left_corner[0]
    angle = np.degrees(np.arctan2(dy, dx))
    
    center = int((left_corner[0] + right_corner[0]) //2), int((left_corner[1] + right_corner[1]) // 2)

    return cv2.getRotationMatrix2D(center, angle, 1.0), center

In [34]:
def process_vid(face_detector, landmark_predictor, input_video_path, output_video_path, fps, width, height, output_size):
    vid_capture = cv2.VideoCapture(input_video_path)
    num_frames = vid_capture.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_count = 0;
    cropped_frames = []
    has_face = True;

    if num_frames >= 29:
        while vid_capture.isOpened():
            ret, frame = vid_capture.read()
            if not ret:
                break
        
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            faces = face_detector(gray)
            if len(faces) == 0:
                has_face = False
                break
            face = faces[0]
            landmarks = landmark_predictor(gray, face)
            
            M, center = get_rotation_matrix(landmarks)

            gray_aligned = cv2.warpAffine(gray, M, (width, height))

            x_min = center[0] - output_size // 2
            y_min = center[1] - output_size // 2
            x_max = x_min + output_size
            y_max = y_min + output_size
            
            cropped_frame = np.zeros((output_size, output_size), dtype=np.uint8)

            src_x_min = max(x_min, 0)
            src_y_min = max(y_min, 0)
            src_x_max = min(x_max, gray.shape[1])
            src_y_max = min(y_max, gray.shape[0])
            
            dst_x_min = max(0, -x_min)
            dst_y_min = max(0, -y_min)
            dst_x_max = dst_x_min + (src_x_max - src_x_min)
            dst_y_max = dst_y_min + (src_y_max - src_y_min)
            
            cropped_frame[dst_y_min:dst_y_max, dst_x_min:dst_x_max] = gray_aligned[src_y_min:src_y_max, src_x_min:src_x_max] 
            cropped_frames.append(cropped_frame)
            frame_count += 1
            if frame_count == 29:
                break

        if has_face:
            output_dir = os.path.dirname(output_video_path)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (output_size, output_size), isColor=False)
            for frame in cropped_frames:
                out.write(frame)
            out.release()
            
    vid_capture.release()
    return has_face

In [ ]:
#Attempt to normalize mouth size, doesn't look great but maybe later can try
#Would go after gray_aligned creation in preprocess
# lip_points = np.array([[landmarks.part(i).x, landmarks.part(i).y] for i in range(48, 61)])
# lip_points = np.hstack([lip_points, np.ones((lip_points.shape[0], 1))])
# rotated_lip_points = M.dot(lip_points.T).T
# x_min, y_min = np.min(rotated_lip_points, axis=0)[:2].astype(int)
# x_max, y_max = np.max(rotated_lip_points, axis=0)[:2].astype(int)

# width_lips = x_max - x_min
# height_lips = y_max - y_min

# scale_x = (output_size * 0.5) / width_lips
# scale_y = (output_size * 0.25) / height_lips

# new_crop_width = output_size // scale_x
# new_crop_height = output_size // scale_y

# x_min = int(center[0] - new_crop_width // 2)
# y_min = int(center[1] - new_crop_height // 2)
# x_max = int(x_min + new_crop_width)
# y_max = int(y_min + new_crop_height)


# cropped_frame = gray_aligned[y_min:y_max, x_min:x_max]
# resized_frame = cv2.resize(cropped_frame, (output_size, output_size))

# out.write(resized_frame)

In [48]:
def test_00001(face_detector, landmark_predictor, source_root, output_root, source_word, mode, input_video):
        input_video_path = os.path.join(source_root, source_word, mode, input_video)
        output_video_path = os.path.join(output_root, source_word, mode, input_video)
        process_vid(face_detector, landmark_predictor, input_video_path, output_video_path, fps=25, width=256, height=256, output_size=64)

def test_whole(face_detector, landmark_predictor, source_root, output_root, source_word, mode):
        start_time = time.time()
        source_folder = os.path.join(source_root, source_word, mode)
        output_video_folder = os.path.join(output_root, source_word, mode)
        added = 0
        for input_video in sorted([file for file in os.listdir(source_folder) if file.endswith(".mp4")]):
                input_video_path = os.path.join(source_folder, input_video)
                output_video_path = os.path.join(output_video_folder, input_video)
                has_face = process_vid(face_detector, landmark_predictor, input_video_path, output_video_path, fps=25, width=256, height=256, output_size=64)
                if has_face:
                        added += 1
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"Finished Processing {source_word} folder in {elapsed: .2f} seconds")
        return added
        

def run_all(face_detector, landmark_predictor, source_root, output_root, mode):
        start_time = time.time()
        source_folders = [folder for folder in sorted(os.listdir(source_root)) if folder != '.DS_Store']

        for source_word in source_folders:
                print(f"Processing word {source_word}...")
                source_folder = os.path.join(source_root, source_word, mode)
                output_video_folder = os.path.join(output_root, source_word, mode)
                for input_video in sorted([file for file in os.listdir(source_folder) if file.endswith(".mp4")]):
                        input_video_path = os.path.join(source_folder, input_video)
                        output_video_path = os.path.join(output_video_folder, input_video)
                        process_vid(face_detector, landmark_predictor, input_video_path, output_video_path, fps=25, width=256, height=256, output_size=64)

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Done in {elapsed_time: .2f} seconds')

In [47]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
source_root = 'selected_mp4_files'
source_word = 'CHINA'
mode = 'test'
input_video = 'ABOUT_00002.mp4'
output_root = 'preprocessed_25'
#This notebook is used to preprocess the mp4's 
#It 1) centers, 2) horizontally aligns the mp4 around the lips center
#3) Grayscale's the mp4, 4) Crops it to a 64x64 mp4
#Then, it creates a new folder called "processed_selected_mp4_files" with the same
#Structure as the lipread_mp4 data to store the new files
#I ran it on mac so it might work differently on windows or something. 
#test_ABOUT_00001 preprocesses just ABOUT_00001.mpy
#test_ABOUT_whole preprocesses the entire ABOUT train data
#run_all preprocesses all the Words train set
class_names = sorted(os.listdir(source_root))
class_names = [name for name in class_names if name != '.DS_Store' and name > 'IRELAND']
# chosen_names = np.random.choice(class_names, 25)
print(class_names)
# run_all(face_detector, landmark_predictor, source_root, output_root, 'train')
# run_all(face_detector, landmark_predictor, source_root, output_root, 'test')

['ISLAMIC', 'ISSUE', 'ISSUES', 'ITSELF', 'JAMES', 'JUDGE', 'JUSTICE', 'KILLED', 'KNOWN', 'LABOUR', 'LARGE', 'LATER', 'LATEST', 'LEADER', 'LEADERS', 'LEADERSHIP', 'LEAST', 'LEAVE', 'LEGAL', 'LEVEL', 'LEVELS', 'LIKELY', 'LITTLE', 'LIVES', 'LIVING', 'LOCAL', 'LONDON', 'LONGER', 'LOOKING', 'MAJOR', 'MAJORITY', 'MAKES', 'MAKING', 'MANCHESTER', 'MARKET', 'MASSIVE', 'MATTER', 'MAYBE', 'MEANS', 'MEASURES', 'MEDIA', 'MEDICAL', 'MEETING', 'MEMBER', 'MEMBERS', 'MESSAGE', 'MIDDLE', 'MIGHT', 'MIGRANTS', 'MILITARY', 'MILLION', 'MILLIONS', 'MINISTER', 'MINISTERS', 'MINUTES', 'MISSING', 'MOMENT', 'MONEY', 'MONTH', 'MONTHS', 'MORNING', 'MOVING', 'MURDER', 'NATIONAL', 'NEEDS', 'NEVER', 'NIGHT', 'NORTH', 'NORTHERN', 'NOTHING', 'NUMBER', 'NUMBERS', 'OBAMA', 'OFFICE', 'OFFICERS', 'OFFICIALS', 'OFTEN', 'OPERATION', 'OPPOSITION', 'ORDER', 'OTHER', 'OTHERS', 'OUTSIDE', 'PARENTS', 'PARLIAMENT', 'PARTIES', 'PARTS', 'PARTY', 'PATIENTS', 'PAYING', 'PEOPLE', 'PERHAPS', 'PERIOD', 'PERSON', 'PERSONAL', 'PHONE', 'PLA

In [46]:
for name in class_names:
    test_whole(face_detector, landmark_predictor, source_root, output_root, name, 'test')
    test_whole(face_detector, landmark_predictor, source_root, output_root, name, 'train')

['SIDES' 'PLACE' 'LEAVE' 'SHOULD' 'HAPPENING' 'FAMILIES' 'SENIOR' 'LIVES'
 'STAGE' 'COMMUNITY' 'EDUCATION' 'TALKING' 'HOUSING' 'GETTING' 'LOOKING'
 'WEEKS' 'FOUND' 'AGAIN' 'SERIOUS' 'INQUIRY' 'WITHIN' 'WHILE' 'LIVING'
 'TONIGHT' 'PERSON']
Finished Processing SIDES folder in  16.78 seconds
Finished Processing SIDES folder in  64.33 seconds
Finished Processing PLACE folder in  15.32 seconds
Finished Processing PLACE folder in  66.03 seconds
Finished Processing LEAVE folder in  14.63 seconds
Finished Processing LEAVE folder in  64.60 seconds
Finished Processing SHOULD folder in  15.07 seconds
Finished Processing SHOULD folder in  64.73 seconds
Finished Processing HAPPENING folder in  15.98 seconds
Finished Processing HAPPENING folder in  64.08 seconds
Finished Processing FAMILIES folder in  14.94 seconds
Finished Processing FAMILIES folder in  59.79 seconds
Finished Processing SENIOR folder in  14.66 seconds
Finished Processing SENIOR folder in  64.42 seconds
Finished Processing LIVES fol